In [54]:
import os
import shutil
import random
import cv2

# Paths
label_dir = r"G:\AgriTech\agri_projects\3-Weed_Detection\Weed-Detection\v11\new_dataset\split_data\labels\test"
source_img_dir = r"G:\AgriTech\agri_projects\3-Weed_Detection\Weed-Detection\v11\new_dataset\split_data\images\test"
target_dir = r"G:\AgriTech\agri_projects\3-Weed_Detection\Weed-Detection\v11\test_video"
video_path = r"G:\AgriTech\agri_projects\3-Weed_Detection\Weed-Detection\v11\weed_detection_video.mp4"

os.makedirs(target_dir, exist_ok=True)

# Step 1 & 2: Collect all eligible label files
eligible_files = []
for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), "r") as f:
            classes = set(line.split()[0] for line in f.readlines())
            if len(classes) >= 3:
                eligible_files.append(label_file)

# Step 3: Randomly select 30 files (or fewer if not enough)
selected_files = random.sample(eligible_files, min(50, len(eligible_files)))

# Step 4: Copy selected images
count = 0
for label_file in selected_files:
    img_name = label_file.replace(".txt", ".jpeg")  # adjust extension if needed
    src_path = os.path.join(source_img_dir, img_name)
    dst_path = os.path.join(target_dir, img_name)
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)
        count += 1

print(f"Total copied images: {count}")

# Create video from copied images
images = [img for img in os.listdir(target_dir) if img.endswith(".jpeg")]
images.sort()

if images:
    first_image_path = os.path.join(target_dir, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 1  # 1 frame per second
    video = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

    for image in images:
        img_path = os.path.join(target_dir, image)
        frame = cv2.imread(img_path)
        video.write(frame)

    video.release()
    print(f"Video saved to {video_path}")

Total copied images: 50
Video saved to G:\AgriTech\agri_projects\3-Weed_Detection\Weed-Detection\v11\weed_detection_video.mp4


In [55]:
from ultralytics import YOLO
import cv2

# Load model
model = YOLO(r"G:/AgriTech/agri_projects/3-Weed_Detection/Weed-Detection/v11/runs/detect/train/weights/best.pt")

# Input and output video paths
input_video_path = r"G:/AgriTech/agri_projects/3-Weed_Detection/Weed-Detection/v11/weed_detection_video.mp4"
output_video_path = r"G:/AgriTech/agri_projects/3-Weed_Detection/Weed-Detection/v11/weed_detection_output.mp4"

# Open video
cap = cv2.VideoCapture(input_video_path)

# Get video properties
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Define colors for 8 classes
COLORS = [
    (255, 0, 0),        # Bright Red
    (0, 255, 0),        # Bright Green
    (0, 0, 255),        # Bright Blue
    (255, 255, 0),      # Bright Yellow
    (255, 0, 255),      # Magenta
    (0, 255, 255),      # Cyan
    (128, 0, 128),      # Purple
    (255, 165, 0)       # Orange
]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model(frame, imgsz=832, conf=0.60)[0]

    # Draw boxes and labels
    for box in results.boxes:
        cls_id = int(box.cls.cpu().numpy()[0])  # get scalar
        label = model.names[cls_id]
        color = COLORS[cls_id % len(COLORS)]

        # Get coordinates - take the first row (4 elements)
        x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)


    # Write frame to output video
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Processed video saved to {output_video_path}")


0: 480x832 1 Moti Dudhi, 12.2ms
Speed: 11.2ms preprocess, 12.2ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 832)

0: 480x832 1 Kena, 1 Little Mallow, 10.8ms
Speed: 4.2ms preprocess, 10.8ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 832)

0: 480x832 2 Kenas, 3 Lamb's Quarterss, 1 Choti Dudhi, 9.7ms
Speed: 5.9ms preprocess, 9.7ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 832)

0: 480x832 4 Lamb's Quarterss, 1 Choti Dudhi, 1 Dwarf Cassia, 10.8ms
Speed: 5.5ms preprocess, 10.8ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 832)

0: 480x832 2 Kenas, 2 Little Mallows, 19.2ms
Speed: 4.8ms preprocess, 19.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 832)

0: 480x832 3 Lamb's Quarterss, 1 Little Mallow, 10.4ms
Speed: 5.0ms preprocess, 10.4ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 832)

0: 480x832 3 Kenas, 3 Lamb's Quarterss, 1 Little Mallow, 9.0ms
Speed: 7.6ms preprocess, 9.0ms inference, 2